In [18]:
import os
import json
import pandas as pd
from sklearn.utils import shuffle
import random

In [19]:
import re
def removecomma(st):
    new_st = re.sub(",", "", st)
    return new_st

In [20]:
working_path = os.path.join(os.getcwd(), "final_data")
fact_desc = dict()
fact_loc = os.path.join(working_path, "fact_sentences")
fact_list = os.listdir(fact_loc)

In [21]:
for x in fact_list:
    name = x[:-4]
    f = open(os.path.join(fact_loc, x), "r")
    fact_desc[name] = removecomma(f.read())
    #break
    

In [22]:
len(fact_desc)

18847

In [23]:
statute_loc = os.path.join(working_path, "charge_sentence")
statute_desc = dict()
statute_list = []

In [24]:
f = open(os.path.join(statute_loc, 'key.json'))
statute_desc_file = json.load(f)

In [25]:
statute_desc_file

{'The Indian Penal Code': '186045.json',
 'Code of Criminal Procedure': '198063.json',
 'Code of Civil Procedure': '190805.json',
 'Land Acquisition Act': '201330.json',
 'Arms Act': '195954.json',
 'Industrial Disputes Act': '194954.json',
 'Negotiable Instruments Act': '188126.json'}

In [26]:
f = open(os.path.join(working_path, "section_case_num.json"))
statute_considered_list = json.load(f)

In [27]:
convert ={  'Code of Criminal Procedure': "Code of Criminal Procedure, 1973_",
            'The Indian Penal Code' : "Indian Penal Code, 1860_",
            "Code of Civil Procedure" : "Code of Civil Procedure, 1882_",
            "Land Acquisition Act" : "Land Acquisition Act, 1894_",
            "Arms Act": "Arms Act, 1959_",
            "Industrial Disputes Act": "Industrial Disputes Act, 1947_",
            "Negotiable Instruments Act" : "Negotiable Instruments Act, 1881_"
        }

for key, value in statute_desc_file.items():
    key = convert[key]
    f = open(os.path.join(statute_loc, value))
    desc_file = json.load(f)
    for num, content in desc_file.items():
        name = key+num
        if name in statute_considered_list:
            statute_list.append(name)
            statute_desc[name] = removecomma(content['title'])+removecomma(content['text'])

print (len(statute_desc), len(statute_list))


41 41


In [28]:
type(statute_desc)

dict

In [29]:
fact_file_loc = os.path.join(working_path, "case_sections.json")
f = open(fact_file_loc)
result_data = json.load(f)

len (result_data.items())


18847

In [30]:
def train_test_split(data):
    random.seed(42)
    test = random.sample(data, int(len(data) * 0.3))
    #print(set(train))
    train =set(data) - set(test)
    #print(test)
    return train, test

In [31]:
type(statute_desc)

dict

In [32]:
train_df = pd.DataFrame(columns= ['Query', 'Statute', 'Result'])
test_df = pd.DataFrame(columns= ['Query', 'Statute', 'Result'])

In [33]:
count =0
for q_id, query_desc  in fact_desc.items():
    pos = []
    neg = []
    

    for statute_id in statute_list:
        if(statute_id in result_data[q_id]):
            pos.append(statute_id)
        else:
            neg.append(statute_id)

    if(len(pos)==0):
        continue    

    pos_train , pos_test = train_test_split(pos)
    neg_train, neg_test = train_test_split(neg)

    print(q_id, len(pos_train), len(pos_test))

    if(len(pos_train)==0):
        continue 
    
    time_of_append = int(len(neg_train)/len(pos_train))+2

    for s_id in pos_train:
        count = count+1
        data = {'Query': query_desc, 'Statute': statute_desc[s_id], 'Result': 1}
        for _ in range(time_of_append):
            train_df = train_df.append(data, ignore_index= True)

    for s_id in neg_train:
        data = {'Query': query_desc, 'Statute': statute_desc[s_id], 'Result': 0}
        train_df = train_df.append(data, ignore_index= True)

    for s_id in pos_test:
        data = {'Query': query_desc, 'Statute': statute_desc[s_id], 'Result': 1}
        test_df = test_df.append(data, ignore_index= True)

    for s_id in neg_test:
        data = {'Query': query_desc, 'Statute': statute_desc[s_id], 'Result': 0}
        test_df = test_df.append(data, ignore_index= True)

    
train_df = shuffle(train_df)
test_df = shuffle(test_df) 


1970_S_262 1 0
2008_S_160 1 0
2003_K_96 5 2
2001_L_33 3 1
2017_I_39 2 0
1971_N_6 4 1
2008_S_606 4 1
2010_D_20 2 0
2002_S_649 3 0
2003_S_560 3 0
2010_S_458 3 1
2003_K_82 2 0
2008_I_47 3 1
2005_A_92 1 0
1968_R_21 2 0
2009_M_165 1 0
1997_S_627 2 0
2009_S_275 6 2
2008_R_17 5 1
2013_M_94 16 6
2003_S_548 1 0
1970_M_26 1 0
1978_T_23 1 0
2009_S_507 4 1
2014_S_193 3 1
2016_G_28 1 0
1982_H_11 1 0
1997_C_399 6 2
2017_R_69 1 0
1997_D_87 2 0
1997_S_633 2 0
2000_S_781 3 1
1997_J_41 3 0
1980_S_217 1 0
2007_S_726 4 1
2009_S_1291 1 0
2014_S_178 5 1
2008_K_209 4 1
1967_S_108 1 0
2008_N_99 1 0
1991_S_224 1 0
2004_S_504 1 0
2002_K_55 3 1
2017_S_372 6 2
2015_N_63 2 0
2004_S_31 1 0
1999_B_4 1 0
2006_S_341 3 0
2008_S_835 3 0
1999_P_58 1 0
2015_G_53 1 0
2007_R_9 2 0
1997_V_18 2 0
1991_R_25 4 1
2012_M_57 1 0
2009_S_1050 4 1
2001_H_57 1 0
1995_S_506 1 0
2003_S_776 3 0
2001_S_1297 2 0
1997_R_150 3 1
1996_A_176 1 0
2002_S_305 3 1
2016_C_58 2 0
1997_S_431 3 1
1996_S_718 1 0
2000_S_568 1 0
2014_S_391 7 2
2003_A_124

1996_U_76 1 0
2014_S_345 1 0
2003_T_76 3 1
1990_J_33 4 1
2001_M_354 1 0
2011_B_56 1 0
2009_S_896 2 0
1994_K_45 3 1
2009_H_38 1 0
2007_S_484 2 0
2010_B_42 5 2
2017_A_62 1 0
1996_S_653 2 0
1995_S_317 6 2
2017_A_76 3 1
2007_S_490 1 0
2009_M_204 3 1
1991_S_186 5 1
2009_S_1127 5 1
2014_S_92 3 0
1962_R_3 1 0
2008_K_197 1 0
2016_S_16 1 0
2011_S_438 4 1
1995_B_49 3 0
1996_S_1073 3 0
2002_S_266 1 0
1977_S_198 3 1
2001_D_54 1 0
1997_S_546 3 0
2009_S_114 3 0
2006_G_74 2 0
2003_D_102 1 0
2008_S_1201 1 0
2000_M_70 1 0
2002_S_272 3 0
2013_S_533 1 0
1997_S_234 1 0
2007_G_74 2 0
1956_H_4 3 0
2007_S_321 4 1
2014_S_219 3 0
2004_S_671 1 0
2009_S_855 5 2
1992_S_401 2 0
2013_B_39 3 0
2012_S_197 1 0
1975_T_46 6 2
2002_S_299 2 0
1977_S_173 1 0
1999_V_12 4 1
2001_S_905 5 2
2002_K_127 3 1
2004_R_66 3 1
2012_S_183 7 3
2009_S_699 3 1
2016_S_474 3 0
2003_C_32 5 1
2013_S_282 4 1
1989_F_3 1 0
2009_S_869 2 0
1996_B_32 3 1
1976_S_272 3 0
2013_K_21 5 2
2017_A_89 6 2
2014_S_231 3 0
2016_S_448 6 2
2004_S_659 1 0
2013_B_

1990_S_269 4 1
2016_S_317 1 0
1992_S_376 3 1
2007_S_456 2 0
2002_S_288 2 0
1996_U_175 1 0
2009_S_92 3 0
2008_A_126 2 0
2011_T_57 3 0
2017_B_36 2 0
2009_S_475 1 0
1994_S_69 1 0
2005_P_105 3 1
1993_K_31 2 0
2006_S_596 3 0
1977_B_28 2 0
1971_H_38 1 0
2000_S_618 5 1
2010_H_32 1 0
2011_S_159 5 2
2010_S_270 3 0
1994_S_205 1 0
1972_P_4 1 0
1995_A_35 3 0
1992_P_49 1 0
1956_J_3 1 0
2010_S_502 3 0
1985_U_57 2 0
1979_V_20 3 0
1983_S_184 1 0
1992_K_19 1 0
2010_M_174 4 1
2009_K_50 2 0
2003_S_374 5 1
1989_S_93 1 0
1997_A_107 3 0
1971_H_6 3 0
1994_T_48 4 1
1979_M_64 1 0
2007_D_20 3 1
2009_R_128 1 0
2003_R_173 1 0
2003_S_360 1 0
2011_S_171 3 1
2003_S_406 2 0
1982_D_17 1 0
1993_P_61 2 0
1984_S_137 3 0
2012_S_347 5 2
1973_S_126 2 0
2008_S_947 3 0
2003_G_53 3 0
1993_S_513 4 1
2013_T_38 2 0
2008_G_177 1 0
2006_S_555 3 0
2001_S_1118 3 0
2017_S_214 3 0
2007_S_668 3 0
2004_S_338 5 2
2011_S_89 1 0
2008_S_953 5 2
1971_S_97 3 1
2015_S_451 3 1
1999_N_16 8 3
1996_S_497 2 0
2009_K_93 1 0
1996_S_483 3 0
2009_K_87 1

2007_S_681 2 0
1993_S_288 2 0
2003_S_376 3 1
2011_S_167 2 0
1994_M_117 1 0
2012_S_351 2 0
1993_P_77 3 0
1973_S_130 2 0
1970_H_12 1 0
1983_S_192 1 0
2004_B_6 5 2
2012_S_345 3 0
2004_C_109 1 0
1995_S_674 4 1
1988_S_91 2 0
1961_H_4 1 0
2003_S_362 2 0
2011_S_173 4 1
1995_S_112 1 0
2010_H_18 3 0
1994_M_103 1 0
2016_P_54 1 0
2010_S_272 5 2
1970_S_56 3 0
2013_I_5 5 2
2006_S_594 1 0
2008_S_1004 1 0
1972_S_225 2 0
2017_P_68 4 1
2012_S_379 4 1
2003_S_438 3 0
2000_I_13 1 0
2001_S_527 3 0
2008_S_90 7 2
2001_S_1442 1 0
2007_S_440 2 0
1996_L_24 1 0
2015_S_56 1 0
1976_S_249 5 1
2017_A_4 5 1
2001_S_902 3 1
1980_S_165 1 0
2007_S_332 3 0
2012_S_184 1 0
2014_S_222 3 0
2008_B_226 3 1
1998_S_307 2 0
2003_J_11 2 0
2010_J_7 1 0
2004_I_31 3 0
2007_S_468 2 0
2008_L_7 1 0
1977_S_148 1 0
1976_S_261 2 0
1997_B_35 3 0
1991_H_25 3 1
2014_S_236 3 1
2008_S_560 5 1
1997_P_82 1 0
2017_T_30 1 0
2016_A_71 4 1
2000_S_342 1 0
1996_S_654 1 0
1987_S_115 5 2
2009_M_217 1 0
2015_S_95 1 0
1970_S_110 2 0
2011_S_417 4 1
1978_N_1 

2004_M_299 1 0
2010_K_13 3 1
2007_S_397 2 0
2005_K_122 3 0
2013_S_208 5 1
2003_U_194 1 0
1997_U_17 1 0
2004_S_758 2 0
1992_S_528 1 0
1993_S_167 1 0
2006_S_647 4 1
1989_R_60 2 0
2002_H_72 1 0
2008_C_87 1 0
1987_A_61 3 0
2017_S_674 6 2
2005_S_465 3 1
2004_K_46 3 0
2014_V_28 2 0
2009_S_940 2 0
2006_M_225 3 0
1988_R_8 6 2
2000_S_293 1 0
2003_S_36 1 0
2014_M_50 3 1
2016_S_213 4 1
2001_K_204 3 0
2013_R_15 2 0
1996_S_949 1 0
2002_S_398 5 1
2014_D_60 2 0
2006_M_231 1 0
2007_L_20 2 0
2001_G_9 1 0
1995_G_24 1 0
2003_J_1 1 0
1997_S_335 3 1
2009_C_44 2 0
1992_J_29 5 2
1975_S_192 1 0
2010_S_604 4 1
1998_T_89 4 1
2011_R_52 1 0
1966_K_23 1 0
2014_S_481 3 1
2003_A_81 3 0
2008_S_1300 3 1
1996_A_114 3 1
1994_S_117 5 1
2013_S_426 5 2
1997_S_321 2 0
2006_K_29 1 0
1973_S_208 3 0
1972_S_121 3 0
2009_S_773 2 0
1996_R_233 1 0
1992_J_15 1 0
2007_B_31 3 0
1995_G_18 1 0
2008_S_472 3 1
1995_S_216 1 0
2017_V_53 4 1
2006_B_25 2 0
2007_S_585 1 0
2004_P_116 5 1
2009_M_311 2 0
2007_B_25 1 0
2011_U_73 1 0
1996_N_90 1 0

...

2003_S_451 1 0
2006_D_63 2 0
2008_V_114 5 2
2010_S_35 6 2
1984_R_21 1 0
2001_S_200 1 0
1996_S_92 1 0
2009_S_344 3 0
2010_S_227 1 0
1997_S_86 3 1
1994_S_252 4 1
2016_B_61 10 3
1997_S_702 1 0
2011_A_41 3 0
2008_M_341 2 0
1994_T_23 1 0
2008_E_16 3 0
1995_T_37 3 0
1995_S_392 2 0
2004_S_623 1 0
2004_N_51 1 0
2013_P_37 1 0
1998_S_408 1 0
2004_R_20 4 1
2004_R_34 1 0
2004_N_45 3 1
2004_S_637 5 1
2002_D_55 2 0
2016_S_426 2 0
1976_H_5 1 0
2015_S_176 2 0
2007_S_401 3 1
2009_M_295 4 1
1992_S_321 1 0
2007_S_429 5 1
1996_B_147 2 0
2013_K_73 1 0
2016_S_87 2 0
2002_S_591 2 0
1986_L_8 2 0
2014_S_277 1 0
2005_S_522 2 0
1990_T_15 1 0
1997_K_44 1 0
1998_S_346 1 0
1996_B_74 1 0
2001_C_219 3 1
2003_V_21 1 0
1992_H_23 5 1
2004_S_186 2 0
2017_A_30 1 0
1996_S_601 1 0
2001_S_980 2 0
2009_S_12 7 3
1995_S_351 1 0
1986_S_269 2 0
2003_S_647 1 0
2008_S_535 1 0
2014_S_288 3 0
1961_D_30 1 0
2009_S_608 5 2
1992_S_484 5 1
2017_A_24 1 0
1995_S_345 2 0
2003_S_135 1 0
1970_S_145 1 0
2003_L_8 1 0
2017_S_78 1 0
1970_K_16 3 0

2003_S_693 2 0
2009_S_810 5 1
2004_S_634 1 0
2015_S_175 3 0
2009_S_804 3 0
2004_S_620 1 0
1995_T_110 1 0
2004_R_23 1 0
1990_S_37 1 0
1995_S_391 2 0
1997_K_124 4 1
2012_P_20 3 0
1992_S_336 2 0
2004_S_146 1 0
2004_D_123 2 0
1990_S_201 2 0
1995_T_138 4 1
1980_R_28 3 1
1989_P_41 3 0
2003_C_77 1 0
1998_D_23 2 0
1997_B_77 5 1
2002_S_592 5 2
1989_P_55 3 0
2014_S_28 5 2
2005_S_535 3 1
2015_T_35 2 0
2008_S_1293 5 1
1978_G_2 3 0
1998_S_351 3 0
2017_A_27 1 0
1995_S_346 3 0
2003_S_136 1 0
2009_S_1176 3 1
2006_N_29 1 0
1977_S_65 3 0
2008_S_536 5 1
2010_A_106 3 0
1992_H_20 1 0
2009_M_241 5 1
2000_S_472 2 0
2007_N_29 5 2
2008_S_1244 1 0
2007_R_70 2 0
2018_U_13 3 1
2011_R_108 1 0
2018_R_9 3 0
1997_K_84 2 0
2001_M_35 3 1
2007_G_25 2 0
2003_M_99 2 0
1996_D_101 1 0
2012_Y_1 1 0
2009_S_623 2 0
1992_A_38 7 2
2003_S_678 3 1
1983_G_12 4 1
1996_S_158 1 0
2016_S_47 1 0
2003_M_226 3 1
2013_S_576 1 0
1997_S_517 2 0
2010_P_73 2 0
1971_B_25 3 0
2009_D_96 1 0
1993_S_619 3 0
2013_G_70 2 0
2004_S_740 1 0
2009_S_970 3

1996_S_937 5 2
1991_D_29 1 0
2013_R_43 4 1
1981_S_308 3 1
1993_S_119 1 0
2001_A_84 1 0
1954_S_13 1 0
1991_M_25 1 0
1997_G_35 10 3
1998_A_61 1 0
2003_S_781 4 1
1999_S_624 5 1
2005_L_25 3 1
2007_S_262 3 1
2012_R_43 6 2
2004_P_40 1 0
2016_S_245 1 0
2007_S_504 3 1
1988_U_17 2 0
2003_P_130 3 1
2002_S_331 4 1
2010_M_218 3 0
1998_S_294 4 1
1997_S_411 3 0
2008_J_36 3 1
2009_S_725 1 0
1994_R_27 2 0
2002_S_457 1 0
2010_N_61 1 0
2009_S_731 2 0
1953_H_2 1 0
1992_P_3 3 1
2010_U_19 2 0
2006_P_13 1 0
2008_N_185 1 0
2017_S_193 5 1
2001_S_675 2 0
2016_M_55 1 0
1994_G_66 1 0
1997_A_5 5 1
1969_C_24 1 0
1979_B_13 1 0
2011_S_235 1 0
2013_R_80 2 0
1998_K_102 2 0
1978_S_110 2 0
2012_R_80 7 3
2009_S_1070 2 0
2006_S_188 1 0
2000_S_206 3 0
2005_P_97 1 0
1998_M_184 1 0
2000_S_560 2 0
2013_S_458 2 0
2016_S_286 5 2
2008_S_815 3 0
2015_S_259 3 0
1970_B_6 3 0
2009_S_1299 2 0
2010_S_487 3 0
2002_P_31 1 0
1996_K_135 3 0
2008_S_1197 1 0
1993_S_333 1 0
2008_S_1183 4 1
2002_S_682 3 0
1996_V_10 2 0
2012_C_7 1 0
1996_J_61 

2017_G_23 1 0
2012_S_202 2 0
1977_N_21 1 0
2009_S_520 3 1
2011_C_26 1 0
2017_I_36 1 0
1992_R_79 1 0
2000_P_59 2 0
2001_S_464 1 0
2003_M_125 1 0
2005_S_193 2 0
2010_C_26 1 0
2008_M_243 1 0
2011_J_16 1 0
2003_S_209 3 1
2013_S_107 5 2
2002_S_646 7 3
2010_S_457 2 0
1993_S_491 1 0
2005_S_839 3 1
1999_P_80 2 0
2006_O_20 2 0
1971_M_29 3 0
2017_R_72 1 0
1996_C_81 1 0
1994_S_378 1 0
1996_S_501 3 1
1989_L_4 3 1
1993_N_34 6 2
1982_S_66 1 0
2007_H_29 2 0
2008_S_812 3 1
2017_S_427 1 0
2005_S_150 1 0
1993_S_320 5 2
1980_S_218 2 0
2013_D_54 1 0
2012_M_64 4 1
2015_S_262 3 0
1991_S_203 3 0
2004_A_62 3 0
1980_A_41 4 1
1999_S_421 1 0
2006_P_1 1 0
1993_R_92 4 1
1980_S_224 1 0
1953_S_59 3 1
2008_S_190 3 0
2007_S_86 2 0
1996_C_56 2 0
2006_S_428 4 1
1996_U_222 3 0
1992_R_92 3 0
2016_M_46 3 1
2002_S_336 2 0
1979_S_202 2 0
2013_S_311 2 0
2007_B_48 5 2
2001_H_58 1 0
1972_S_170 1 0
2008_S_1437 1 0
2009_M_38 2 0
2013_S_463 4 1
1998_S_287 2 0
2002_S_322 3 0
2007_K_78 1 0
2017_D_62 1 0
2017_S_194 1 0
1996_S_703 5 2

2017_S_632 3 0
1981_B_13 2 0
1982_S_112 2 0
2017_S_626 3 0
1972_G_24 6 2
2005_B_24 2 0
2008_V_80 3 1
1998_S_535 3 0
2002_S_490 3 0
1975_M_20 3 1
1996_S_927 2 0
2007_S_528 2 0
2005_S_351 3 0
2013_N_22 5 1
2007_S_500 1 0
2005_S_379 2 0
1996_R_64 1 0
2006_M_277 5 1
2008_T_104 3 0
2003_S_785 3 1
1989_S_248 4 1
2000_A_94 1 0
2016_S_533 6 2
2015_D_26 3 1
2007_S_514 2 0
2003_P_120 4 1
2018_I_14 5 1
1997_S_575 5 1
2005_S_594 1 0
1993_T_6 1 0
2002_S_527 3 0
2017_S_25 4 1
1983_R_31 2 0
2011_S_379 3 0
1986_S_220 2 0
1977_T_7 4 1
1980_S_186 2 0
1999_H_3 1 0
1992_S_397 3 0
2002_S_269 2 0
1987_S_121 1 0
1969_H_15 1 0
2016_S_19 1 0
1990_S_96 2 0
2004_S_681 3 1
2010_K_55 3 1
2009_S_1100 4 1
1994_A_111 2 0
2000_S_362 1 0
2012_S_173 3 0
2004_W_1 1 0
2011_S_345 3 0
2012_I_8 3 0
2002_C_15 1 0
1999_D_55 3 1
2012_B_22 4 1
2005_R_69 3 1
2007_S_448 4 1
2004_M_220 1 0
2003_J_25 1 0
1996_S_847 3 0
1996_S_1083 3 1
2012_S_198 2 0
2002_J_31 3 1
2009_S_682 1 0
2005_S_543 1 0
1972_B_28 3 1
1998_M_71 1 0
2004_I_11 1 

In [34]:
train_df.to_csv('train2.csv', index=False)
test_df.to_csv('test2.csv', index=False)